# Benchmarks 3: Chunk Size Variation

## Explanation

We compared the performance of tiling artificially generated Zarr data to different chunk sizes. The CMIP6 data provides an excellent real world dataset, but is relatively low resolution. In order to study the impact of higher resolution data, we artificially generated Zarr datastores to explore the relationship between tile generation time and chunk size.

## Dataset Generation

We generated multiple data stores of increasingly fine spatial resolution so the total size of each chunk, with no spatial chunks, is multiplied by a factor of 2, to allow for increased chunk size.

The code to produce the zarr stores are in the tile-benchmarking repo: [01-generate-datasets/generate-fake-data-with-chunks.ipynb](https://github.com/developmentseed/tile-benchmarking/blob/main/01-generate-datasets/generate-fake-data-with-chunks.ipynb).

## Tests

Tests were run via the [tile-benchmarking/02-run-tests/03-chunk-size.ipynb](https://github.com/developmentseed/tile-benchmarking/blob/main/02-run-tests/03-chunk-size.ipynb) notebook.

In [2]:
import pandas as pd
import warnings
import holoviews as hv

pd.options.plotting.backend = "holoviews"

warnings.filterwarnings("ignore")

In [4]:
git_url_path = "https://raw.githubusercontent.com/developmentseed/tile-benchmarking/main/02-run-tests/results-csvs/"
df = pd.read_csv(f"{git_url_path}/03-chunk-size-results.csv")

In [6]:
zooms = range(6)
plt_opts = {"width": 400, "height": 300}

plts = []

for zoom_level in zooms:
    df_level = df[df["zoom"] == zoom_level]
    plts.append(
        df_level.hvplot.box(
            y="time",
            by=["chunk_size_mb"],
            c="chunk_size_mb",
            cmap="Plasma_r",
            ylabel="Time to render (ms)",
            xlabel="Chunk size (MB)",
            legend=False,
            title=f"Zoom level {zoom_level}",
        ).opts(**plt_opts)
    )
hv.Layout(plts).cols(2)

:Layout
   .BoxWhisker.I   :BoxWhisker   [chunk_size_mb]   (time)
   .BoxWhisker.II  :BoxWhisker   [chunk_size_mb]   (time)
   .BoxWhisker.III :BoxWhisker   [chunk_size_mb]   (time)
   .BoxWhisker.IV  :BoxWhisker   [chunk_size_mb]   (time)
   .BoxWhisker.V   :BoxWhisker   [chunk_size_mb]   (time)
   .BoxWhisker.VI  :BoxWhisker   [chunk_size_mb]   (time)

## Interpretation of the Results

It's clear smaller chunk sizes support faster tile rendering, regardless of zoom level. However, the tradeoff will be in the number of chunks. Smaller chunk sizes means more spatial chunks will be required to render the data which can impact performance at low zoom levels. We will take a look at this in the next notebook [Benchmarks: Exploring Spatial Chunk Variation Impacts on Tile Generation](./04-number-of-spatial-chunks).